In [1]:
import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))
from openTable import *

from statistics import mean

In [2]:
############ INPUT

def input_nilai_utbk(event):
    col = ['penalaran', 'kuantitatif', 'umum', 'baca_tulis', 'mat_saintek', 
           'fisika', 'kimia', 'biologi', 'mat_soshum', 'geografi', 'sejarah', 'sosiologi', 'ekonomi']
    columns = []
    values = []
    val = []
    
    for key,value in event.items():
        if key == 'pilihan':
            pass
        else:
            columns.append(str(key))
            values.append(value)
            if key in col:
                val.append(value)
                
    ds_server,ds_koneksi = Connection_2()
                
    columns.append('rerata')
    values.append(mean(val))
    table = 'rasionalisasi_nilai'
    status = to_db(ds_koneksi,table,columns,values)
#     input_pilihan
    table = 'rasionalisasi_pilihan'
    columns = ['userId','jurusanId']
    for i in event['pilihan']:
        values = [event['userId'], i]
        status = to_db(ds_koneksi,table,columns,values)   
      
    ds_koneksi.commit()
    ds_koneksi.close()
    ds_server.stop()
        
def tambah_pilihan(event):
    ds_server,ds_koneksi = Connection_2()
    
    table = 'rasionalisasi_pilihan'
    columns = ['userId','jurusanId']
    for i in event['pilihan']:
        values = [event['userId'], i]
        status = to_db(ds_koneksi,table,columns,values)
        
    ds_koneksi.commit()
    ds_koneksi.close()
    ds_server.stop()
    
############ EDIT
def edit_nilai_utbk(event):
    ds_server,ds_koneksi = Connection_2()
    
    userId = event['userId']
    col = ['penalaran', 'kuantitatif', 'umum', 'baca_tulis', 'mat_saintek', 'fisika', 'kimia', 'biologi', 'mat_soshum', 
           'geografi', 'sejarah', 'sosiologi', 'ekonomi']
    tables = 'rasionalisasi_nilai'
    statement = " WHERE userId = " + str(userId)
    status = update_db(ds_koneksi,tables, event, statement)
    ds_koneksi.commit()
    
    nilai,status = open_table_ds(ds_koneksi,col, tables, statement)
    nilai = [x for x in list(nilai[0][0]) if x is not None]
    rerata = {'rerata':mean(nilai)}
    status = update_db(ds_koneksi,tables, rerata, statement)
    
    ds_koneksi.commit()
    ds_koneksi.close()
    ds_server.stop()
    

########### OUTPUT
def get_result_rasionalisasi(event):
    ds_server,ds_koneksi = Connection_2()

    userId = event['userId']
    jurusan = event['jurusanId']
    col_nilai = ['penalaran', 'kuantitatif', 'umum', 'baca_tulis', 'mat_saintek', 'fisika', 
           'kimia', 'biologi', 'mat_soshum', 'geografi', 'sejarah', 'sosiologi', 'ekonomi']

    col_kampus = ['rerata_penalaran', 'rerata_kuantitatif', 'rerata_umum', 'rerata_baca_tulis', 
                  'rerata_mat_saintek', 'rerata_fisika', 'rerata_kimia', 'rerata_biologi', 'rerata_mat_soshum', 
                  'rerata_geografi', 'rerata_sejarah', 'rerata_sosiologi', 'rerata_ekonomi','tahun_ini_pendaftar','tahun_lalu_kuota','tahun_lalu_peminat']

    statement = ' WHERE userId = ' + str(userId)
    tambah = ' AND jurusanId = ' + str(jurusan)
    kp = ' WHERE jurusanId = ' + str(jurusan)

    data_nilai, status = open_table_ds(ds_koneksi,col_nilai,'rasionalisasi_nilai',statement)
    # data_nilai = pd.DataFrame(data_nilai)
    data_pilihan, status = open_table_ds(ds_koneksi,['ranking','recommendation'],'rasionalisasi_pilihan',statement+tambah)
    # data_pilihan = pd.DataFrame(data_pilihan)
    data_kampus, status = open_table_ds(ds_koneksi,col_kampus,'rasionalisasi_kampus', kp)
    # data_kampus = pd.DataFrame(data_kampus)
    
    ds_koneksi.close()
    ds_server.stop()

    nilai_user = {}
    for i in range(0,len(data_nilai[0])):
        if data_nilai[0][i] is not None:
            nilai_user[str(col[i])] = data_nilai[0][i]

    nilai_jurusan = {}
    for i in range(0,len(data_kampus[0])-3):
        if data_kampus[0][i] is not None:
            nilai_jurusan[col_kampus[i]] = data_kampus[0][i]

    result = {}
    result['nilai_user'] = nilai_user
    result['nilai_jurusan'] = nilai_jurusan
    result['rangking_user'] = data_pilihan[0][0]
    result['jumlah_peserta'] = data_kampus[0][13]
    result['peluang'] = data_pilihan[0][1]
    result['kuota_tahun_ini'] = data_kampus[0][14]
    result['tahun_lalu_peminat'] = data_kampus[0][15]    
    
    return result